# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

In [1]:
# Your code goes here
import initial_check as ch
import pandas as pd

%load_ext autoreload
%autoreload 2

In [2]:
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"

data1 = pd.read_csv(url)

In [3]:
data1.head()

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN


In [4]:
ch.check(data1)

Number of columns: 26 amd rows: 10910

Data types:
Unnamed: 0                         int64
Customer                          object
State                             object
Customer Lifetime Value          float64
Response                          object
Coverage                          object
Education                         object
Effective To Date                 object
EmploymentStatus                  object
Gender                            object
Income                             int64
Location Code                     object
Marital Status                    object
Monthly Premium Auto               int64
Months Since Last Claim          float64
Months Since Policy Inception      int64
Number of Open Complaints        float64
Number of Policies                 int64
Policy Type                       object
Policy                            object
Renew Offer Type                  object
Sales Channel                     object
Total Claim Amount               float64
Vehicl

In [5]:
ch.snake_f(data1)
data1.effective_to_date = pd.to_datetime(data1.effective_to_date)
data1 = ch.drop_col(data1,"unnamed:_0")
data1.head()

C:\Users\Arturo\AppData\Local\Temp\ipykernel_16120\3609758752.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data1.effective_to_date = pd.to_datetime(data1.effective_to_date)


,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
0,DK49336,Arizona,4809.216960,No,Basic,College,2011-02-18,Employed,M,48029,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,KX64629,California,2228.525238,No,Basic,College,2011-01-18,Unemployed,F,0,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2011-02-10,Employed,M,22139,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,XL78013,Oregon,22332.439460,Yes,Extended,College,2011-01-11,Employed,M,49078,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,2011-01-17,Medical Leave,F,23675,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN


In [6]:
ch.check(data1)

Number of columns: 25 amd rows: 10910

Data types:
customer                                 object
state                                    object
customer_lifetime_value                 float64
response                                 object
coverage                                 object
education                                object
effective_to_date                datetime64[ns]
employmentstatus                         object
gender                                   object
income                                    int64
location_code                            object
marital_status                           object
monthly_premium_auto                      int64
months_since_last_claim                 float64
months_since_policy_inception             int64
number_of_open_complaints               float64
number_of_policies                        int64
policy_type                              object
policy                                   object
renew_offer_type                     

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [7]:
data2 = data1[(data1['total_claim_amount'] > 1000) & (data1['response'] == "Yes")]
data2.head()

,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,income,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
189,OK31456,California,11009.130490,Yes,Premium,Bachelor,2011-01-24,Employed,F,51643,...,0.0,1,Corporate Auto,Corporate L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,NaN
236,YJ16163,Oregon,11009.130490,Yes,Premium,Bachelor,2011-01-24,Employed,F,51643,...,0.0,1,Special Auto,Special L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,A
419,GW43195,Oregon,25807.063000,Yes,Extended,College,2011-02-13,Employed,F,71210,...,1.0,2,Personal Auto,Personal L2,Offer1,Branch,1027.200000,Luxury Car,Small,A
442,IP94270,Arizona,13736.132500,Yes,Premium,Master,2011-02-13,Disabled,F,16181,...,0.0,8,Personal Auto,Personal L2,Offer1,Web,1261.319869,SUV,Medsize,A
587,FJ28407,California,5619.689084,Yes,Premium,High School or Below,2011-01-26,Unemployed,M,0,...,0.0,1,Personal Auto,Personal L1,Offer2,Web,1027.000029,SUV,Medsize,A


2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [11]:
data3 = data1[data1['response'] == "Yes"]
data4 = data3.groupby(['policy_type', 'gender'])['total_claim_amount'].mean().reset_index()
data4

,policy_type,gender,total_claim_amount
0,Corporate Auto,F,433.738499
1,Corporate Auto,M,408.582459
2,Personal Auto,F,452.965929
3,Personal Auto,M,457.010178
4,Special Auto,F,453.280164
5,Special Auto,M,429.527942


In [12]:
# Special Auto policies have the highest average total claim amounts with females, while with males personal auto have the highest claims. Corporal 
# Auto policies have the lowest claim amounts on average, but still vary by gender.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [13]:
df_state_counts = data1['state'].value_counts().reset_index()
df_state_counts.columns = ['State', 'Total Customers']
df_filtered_states = df_state_counts[df_state_counts['Total Customers'] > 500]
df_filtered_states


,State,Total Customers
0,California,3552
1,Oregon,2909
2,Arizona,1937
3,Nevada,993
4,Washington,888


4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [ ]:
df_clv_stats = data1.groupby(['education', 'gender'])['customer_lifetime_value'].agg(['max', 'min', 'median']).reset_index()
df_clv_stats

,education,gender,max,min,median
0,Bachelor,F,73225.95652,1904.000852,5640.505303
1,Bachelor,M,67907.27050,1898.007675,5548.031892
2,College,F,61850.18803,1898.683686,5623.611187
3,College,M,61134.68307,1918.119700,6005.847375
4,Doctor,F,44856.11397,2395.570000,5332.462694
5,Doctor,M,32677.34284,2267.604038,5577.669457
6,High School or Below,F,55277.44589,2144.921535,6039.553187
7,High School or Below,M,83325.38119,1940.981221,6286.731006
8,Master,F,51016.06704,2417.777032,5729.855012
9,Master,M,50568.25912,2272.307310,5579.099207


In [16]:
pivot_df = df_clv_stats.pivot_table(index='education', columns='gender', values=['max', 'min', 'median'])
pivot_df

max                    median               \
gender                          F            M            F            M   
education                                                                  
Bachelor              73225.95652  67907.27050  5640.505303  5548.031892   
College               61850.18803  61134.68307  5623.611187  6005.847375   
Doctor                44856.11397  32677.34284  5332.462694  5577.669457   
High School or Below  55277.44589  83325.38119  6039.553187  6286.731006   
Master                51016.06704  50568.25912  5729.855012  5579.099207   

                              min               
gender                          F            M  
education                                       
Bachelor              1904.000852  1898.007675  
College               1898.683686  1918.119700  
Doctor                2395.570000  2267.604038  
High School or Below  2144.921535  1940.981221  
Master                2417.777032  2272.307310

In [17]:
# Customers with a Bachelor's degree have the highest maximum customer life time value.  
# Customers with a High School or Below education level have the highest max customer life time value for males

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

In [19]:
data5 = data1.copy()
data5['month'] = data5['effective_to_date'].dt.month  


df_policies = data5.pivot_table(values='number_of_policies', index='state', columns='month', aggfunc='sum', fill_value=0)
df_policies

month,1,2
state,,
Arizona,3052,2864
California,5673,4929
Nevada,1493,1278
Oregon,4697,3969
Washington,1358,1225


6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

In [21]:
data6 = data1.copy()
data6['month'] = data6['effective_to_date'].dt.month  

state_month_policies = data6.groupby(['state', 'month'])['number_of_policies'].sum().reset_index()

state_totals = state_month_policies.groupby('state')['number_of_policies'].sum().reset_index()
top_states = state_totals.sort_values(by='number_of_policies', ascending=False).head(3)

top_states_data = state_month_policies[state_month_policies['state'].isin(top_states['state'])]
top_states_pivot = top_states_data.pivot(index='state', columns='month', values='number_of_policies').fillna(0)


top_states_pivot

month,1,2
state,,
Arizona,3052,2864
California,5673,4929
Oregon,4697,3969


7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [34]:
# your code goes here

response_counts = data1.groupby('sales_channel')['response'].value_counts().unstack()


response_counts['response_rate'] = (response_counts['Yes'] / (response_counts['Yes'] + response_counts['No'])) * 100
response_rates = response_counts[['response_rate']].reset_index()



response_rates




response,sales_channel,response_rate
0,Agent,19.074550
1,Branch,11.378709
2,Call Center,10.978639
3,Web,11.714097
